# Guide: run async programs

In this guide we will look into an ability of QuantumServerless running asynchronous programs as async jobs.

Let's import all necessary dependencies and create `QuantumServerless` instance.

In [1]:
from quantum_serverless import QuantumServerless, Program, GatewayProvider, Provider
from quantum_serverless.core import ComputeResource

In [2]:
gateway_provider = GatewayProvider(
    username="john",
    password="password123",
    host="http://localhost:8000",
)

serverless = QuantumServerless([gateway_provider])
serverless

<QuantumServerless | providers [gateway-provider]>

> NOTE: important thing to remember is that async jobs are only available with real cluster and will not work in local machine. 

In this example we will have head node running on inside docker on localhost

---

Now we have all prep steps figured out. Next step will be defining script for the job itself.

Inside the script we will be using the same principles that we showed in other tutorials (creating QuantumServerless instance, annotating functions and getting results)

```python
# program.py
from quantum_serverless import QuantumServerless, run_qiskit_remote, get


@distribute_task()
def ultimate():
    return 42


with QuantumServerless().context():
    result = get([
        ultimate()
        for _ in range(100)
    ])
    
print(result)
```

[Program file](./program.py) will be located in the same directory as our notebook.


Let's run our script/job using `run_program` function provided by QuantumServerless.
Run program accepts `Program` instance.

In [12]:
program = Program(
    name="awesome_program",
    entrypoint="program.py",
    working_dir="./",
    description="description of a program",
    version="0.0.1"
)

job = serverless.run_program(program)
job

<Job | 2>

You will get `Job` object as result, which you can query for `status` and `logs` to see status of your job and logs.

In [13]:
job.status()

'SUCCEEDED'

In [14]:
print(job.logs())

Single execution: [0.]
N parallel executions: [array([0.]), array([1.]), array([0.])]



In [15]:
job.result()

'{"single_execution": [0.0], "multiple_executions": [[0.0], [1.0], [0.0]]}'

In [16]:
# recover job by id
recovered_job = serverless.get_job_by_id(job.job_id)

In [17]:
recovered_job.status()

'SUCCEEDED'

In [18]:
recovered_job.result()

'{"single_execution": [0.0], "multiple_executions": [[0.0], [1.0], [0.0]]}'